In [1]:
import time
import os

import IPython.display as ipd
import numpy as np
import pandas as pd

import librosa
from pathlib import Path,PureWindowsPath,PurePosixPath
import torchaudio
import torch
import matplotlib.pyplot as plt
import utils

SAMPLE_RATE = 44100

In [2]:
tracks = pd.read_csv("./fma_metadata/tracks.csv")
genres = pd.read_csv("./fma_metadata/genres.csv")
tracks.columns=tracks.iloc[0] 
tracks.columns.values[0] = "track_id"
tracks.drop([0,1],inplace=True)
tracks.track_id = tracks.track_id.astype(int)

C:\Users\manue\AppData\Local\Temp\ipykernel_8596\1660309712.py:1: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  tracks = pd.read_csv("./fma_metadata/tracks.csv")


In [6]:
def GetGenres(path,genre = 'genre_top'):
    id_list = []
    genre_list = []
    for direc in list(path.iterdir()):
        if not direc.is_file():
            for file in (list(direc.iterdir())):
                id_track = str(file)[-10:-4]
                id_list.append(id_track)
                genre_list.append(tracks.loc[tracks.track_id == int(id_track),genre].values[0])
    return np.asarray(id_list),np.asarray(genre_list)


In [7]:
path = Path("./fma_small")
id_list, genre_list = GetGenres(path)

In [24]:
def CreateSpectrograms(load_path,save_path, transformation = "MEL"):
    id_list = []
    genre_list = []
    if transformation == 'MEL':
        transform = torchaudio.transforms.MelSpectrogram(SAMPLE_RATE,n_fft=2048,hop_length=512)
    else:
        transform = torchaudio.transforms.Spectrogram(SAMPLE_RATE,n_fft=2048,hop_length=512)
    for direc in list(load_path.iterdir()):
        if not direc.is_file():
            for file in (list(direc.iterdir())):
                id_track = str(file)[-10:-4]
                waveform, sample_rate = torchaudio.load(file)
                spec = transform(waveform)
                torch.save(spec, save+"/"+id_track+".pt")
                                                


In [ ]:
save = "./Spectrograms"
CreateSpectrograms(path,save)